# Altimetry

## Import packages

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Time period
start = "1993-01"
stop = "2019-12"

## Define request

In [ ]:
collection_id = "satellite-sea-level-global"
request = {
    "version": "vDT2021",
    "variable": "daily",
    "format": "zip",
}
requests = download.update_request_date(
    request, start=start, stop=stop, stringify_dates=True
)

## Functions to cache

In [ ]:
def compute_extreme(da):
    da_95 = da.chunk({"time": -1}).quantile(q=0.95, dim="time").drop_vars("quantile")
    da_extreme = da.where(da > da_95).mean("time")
    da_extreme.attrs["long_name"] = f"Mean extreme values of {da.attrs['long_name']}"
    da_extreme.attrs["units"] = da.attrs["units"]
    return da_extreme.rename("extreme")


def compute_time_statistics(ds, reduction):
    da = ds["sla"]
    if reduction == "extreme":
        da_reduced = compute_extreme(da)
    else:
        da_reduced = getattr(diagnostics, f"time_weighted_{reduction}")(
            da, weights=False
        )
        da_reduced.attrs = {
            "units": da_reduced.attrs["units"],
            "long_name": f"{reduction.capitalize().replace('_', ' ')} of {da.attrs['long_name']}",
        }
    return da_reduced.to_dataset(name=reduction)


def compute_spatial_weighted_statistics(ds):
    da = ds["sla"]
    ds_diag = diagnostics.spatial_weighted_statistics(da).to_dataset(dim="diagnostic")
    for reduction, da_reduced in ds_diag.data_vars.items():
        da_reduced.attrs = {
            "units": da.attrs["units"],
            "long_name": f"Spatial weighted {reduction} of {da.attrs['long_name']}",
        }
    return ds_diag

## Compute time reductions

In [ ]:
# Kwargs to speed-up IO
xarray_kwargs = {
    "concat_dim": "time",
    "combine": "nested",
    "data_vars": "minimal",
    "coords": "minimal",
    "compat": "override",
    "parallel": True,
}

datasets = []
for reduction in ("mean", "std", "coverage", "extreme", "linear_trend"):
    if reduction in ["extreme", "linear_trend"]:
        # TODO: Work in progress
        # Do not compute extreme and linear trend, it crashes on the VM.
        continue
    print(f"Computing {reduction=}")
    ds = download.download_and_transform(
        collection_id,
        requests,
        chunks={"year": 1},
        transform_func=compute_time_statistics,
        transform_func_kwargs={"reduction": reduction},
        transform_chunks=False,
        **xarray_kwargs,
    )
    datasets.append(ds)
ds_maps = xr.merge(datasets)

## Plot maps

In [ ]:
for var, da in ds_maps.data_vars.items():
    center = 0 if var == "mean" else None
    plot.projected_map(da, projection=ccrs.Robinson(), robust=True, center=center)
    plt.title(f"{da.attrs['long_name']} ({start}, {stop})")
    plt.show()